# Dive Into Football Analytics With TensorFlow Object Detection API  [Link](https://neptune.ai/blog/dive-into-football-analytics-with-tensorflow-object-detection-api)


## Generate Images from Video

In [1]:
# Environment 
import os
## pip install opencv-python 
import cv2

In [6]:
ChelseaManCity = r"C:\Users\yingy\Desktop\CHE_vs_AST_-_First-Half_-_Sky.mp4"
vidcap = cv2.VideoCapture(ChelseaManCity)
count = 0
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(r"C:\Users\yingy\Desktop\image"+"\\"+str(sec)+".jpg", image)     # save frame as JPG file
    return hasFrames
sec = 8*60 # starting from 8*60 seconds of the video
frameRate = 2 #//it will capture image in each 2 second
count=0
success = getFrame(sec)
if not os.path.exists('images'):
    os.makedirs('images')
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)
    if count > 300:
        break

## Labeling on Labelbox
[Project Link](https://app.labelbox.com/projects/clgmw7co50urt070y3evv6o23/overview)

## Export labeled data from Labelbox
[Reference on Export](https://docs.labelbox.com/reference/label-export) 

[Colab for export](https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/label_export/images.ipynb#scrollTo=Ph1D2NethWsU)


In [1]:
# INSTALL pip install labelbox
# ERROR Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
# SOLUTION https://stackoverflow.com/questions/64261546/how-to-solve-error-microsoft-visual-c-14-0-or-greater-is-required-when-inst
import labelbox as lb
import labelbox.types as lb_types

from PIL import Image
import numpy as np
import os
import json

In [2]:
# Add your api key and project
from dotenv import load_dotenv
load_dotenv("labelbox.env")
API_KEY = os.getenv("labelbox_soccer")
PROJECT_ID = os.getenv("labelbox_soccer_project")
client = lb.Client(api_key=API_KEY)
project = client.get_project(PROJECT_ID)

In [9]:
# Method 1
# labels = project.label_generator()

# Old export format:
labels = project.export_labels(download = True)
# this seems to be working with roboflow conversion

# labels can also be exported with `start` and `end` filters
# labels = project.label_generator(start="2020-01-01", end="2020-01-02")
# labels = list(labels)

In [10]:
json_object = json.dumps(labels, indent=4)
with open("export_20230509.json", "w") as outfile:
    outfile.write(json_object)

In [4]:
# Method 2
export_params= {
  "attachments": True,
  "metadata_fields": True,
  "data_row_details": True,
  "project_details": True,
  "performance_details": True
}
# filters= {
#   "last_activity_at": ["2000-01-01 00:00:00", "2050-01-01 00:00:00"]
# }
export_task = project.export_v2(params=export_params)# , filters=filters)
export_task.wait_till_done()

if export_task.errors:
    print(export_task.errors)

export_json = export_task.result

In [8]:
json_object = json.dumps(export_json, indent=4)
with open("export_20230509.json", "w") as outfile:
    outfile.write(json_object)